In [1]:
%pip install xgboost

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [2]:
import pandas as pd
import numpy as np

In [3]:
# THIS IS FOR DATA CLEANING - all the stuff should be here
df_train = pd.read_csv('train.csv')
df_test = pd.read_csv('test.csv')

df_train = df_train[df_train['intake_type'] != 'Wildlife']
df_train = df_train.dropna(subset=['age_upon_intake'])

In [4]:
def drop_cols(df):
    cols_to_drop = ['id', 'name', 'date_of_birth', 'outcome_time', 'found_location']
    existing_cols = [col for col in cols_to_drop if col in df.columns]
    df = df.drop(columns=existing_cols)
    return df

In [5]:
def clean_intake_time(df):
    # Intake Time
    # Convert string timestamps to UNIX timestamp
    dt_series = pd.to_datetime(df['intake_time'], errors='coerce')
    df['intake_time'] = dt_series.astype('int64') // (10 ** 9)
    return df

# Age Upon Intake
def convert_age(age_str):
    """
    Convert age strings to years.
    Expected format: "<number> <unit>" e.g., "2 years", "8 months", "3 weeks", "15 days"
    """
    if not isinstance(age_str, str):
        return None

    parts = age_str.split()
    if len(parts) != 2:
        return None

    try:
        num = float(parts[0])
    except ValueError:
        return None

    unit = parts[1].lower()
    if "year" in unit:
        return num
    elif "month" in unit:
        return num / 12
    elif "week" in unit:
        return num / 52
    elif "day" in unit:
        return num / 365
    else:
        return None

def clean_age_and_sex_upon_intake(df):
    df.loc[df['sex_upon_intake'].isna(), 'sex_upon_intake'] = 'Unknown'
    df['age_upon_intake'] = df['age_upon_intake'].apply(convert_age)
    df.loc[df['age_upon_intake'] < 0, 'age_upon_intake'] = 0
    return df

# Breed
def clean_breed(df):
# Create is_mix column
    df['is_mix'] = df['breed'].str.contains('mix', case=False, na=False).astype(int)
    # remove mix from all breeds
    df['breed'] = df['breed'].str.replace(' mix', '', case=False)
    return df

In [6]:
color_group_map = {
  'blue tabby':       'gray tabby',
  'silver tabby':     'gray tabby',
  'silver':           'gray',
  'blue':             'gray',
  'orange tabby':     'orange',
  'orange tiger':     'orange',
  'red':              'orange',
  'red tabby':        'orange',
  'red tick':         'orange',
  'yellow':           'orange',
  'tan':              'cream',
}

def clean_color(df):
  # lowercase
  df['color'] = df['color'].str.lower().str.strip()

  # feature engineering -> primary color 
  df['primary_color'] = df['color'].astype(str).apply(
      lambda x: x.split('/')[0].strip() if '/' in x else x.strip()
  )

  # simplify synonymous colors if in map
  df['primary_color'] = df['primary_color'].map(color_group_map).fillna(df['primary_color'])

  df = df.drop(columns=['color'])
  return df


def freq_encode(df, col):
  # count frequencies
  freq_series = df[col].value_counts()

  # map frequencies back to the original column, replacing values
  df[col] = df[col].map(freq_series)

  return df

In [ ]:
def clean_intake_cond(df):
    # Known mappings
    df['intake_condition'] = df['intake_condition'].replace({'Unknown': 'Unknown Condition / Other', 'Other': 'Unknown Condition / Other'})
    df['intake_condition'] = df['intake_condition'].replace({'Space': 'Unknown Condition / Other'})
    df['intake_condition'] = df['intake_condition'].replace({'Behavior': 'Normal / Behavior', 'Normal': 'Normal / Behavior'})
    df['intake_condition'] = df['intake_condition'].replace({'Neonatal': 'Nursing / Neonatal', 'Nursing': 'Nursing / Neonatal'})
    df['intake_condition'] = df['intake_condition'].replace({'Neurologic': 'Med Urgent', 'Agonal': 'Med Urgent', 'Parvo': 'Med Urgent'})
    df['intake_condition'] = df['intake_condition'].replace({'Agonal': 'Med Urgent / Neurological'})
    df['intake_condition'] = df['intake_condition'].replace({'Congenital': 'Sick'})
    
    return df


In [9]:
def clean_data(df):
    df = drop_cols(df)
    print('dropped columns')
    df = clean_intake_time(df)
    print('cleaned intake time')
    df = clean_intake_cond(df)
    print('cleaned intake condition')
    df = clean_age_and_sex_upon_intake(df)
    print('cleaned age and sex')
    df = clean_breed(df)
    print('cleaned breed')
    df = clean_color(df)
    print('cleaned color')
    # df = clean_animal_type(df)
    # print('cleaned animal type')
    return df

In [10]:
from sklearn.preprocessing import OneHotEncoder
def encode_columns(df):

    encoder = OneHotEncoder(handle_unknown='ignore', sparse=False)
    encoder.fit(df[['category_column']])

    dummies = pd.get_dummies(df['intake_type'])
    df = df.drop('intake_type', axis=1)
    df = pd.concat([df, dummies], axis=1)

    dummies = pd.get_dummies(df['intake_condition'])
    df = df.drop('intake_condition', axis=1)
    df = pd.concat([df, dummies], axis=1)

    dummies = pd.get_dummies(df['sex_upon_intake'])
    df = df.drop('sex_upon_intake', axis=1)
    df = pd.concat([df, dummies], axis=1)
    return df

In [11]:
df_train = clean_data(df_train)
# df_train = encode_columns(df_train)
label_column = df_train.pop('outcome_type')
df_train.insert(df_train.shape[1], 'outcome_type', label_column)    
df_train.head()

dropped columns
cleaned intake time
cleaned intake condition
cleaned age and sex
cleaned breed
cleaned color


,intake_time,intake_type,intake_condition,animal_type,sex_upon_intake,age_upon_intake,breed,is_mix,primary_color,outcome_type
0,1436101140,Stray,Normal / Behavior,Dog,Spayed Female,8.000000,English Springer Spaniel,0,white,Return to Owner
1,1460659380,Stray,Normal / Behavior,Dog,Intact Male,0.916667,Basenji,1,sable,Return to Owner
2,1652314980,Public Assist,Normal / Behavior,Cat,Neutered Male,2.000000,Domestic Shorthair,0,orange,Transfer
3,1487421960,Owner Surrender,Normal / Behavior,Dog,Neutered Male,2.000000,Labrador Retriever,1,chocolate,Return to Owner
4,1555408380,Public Assist,Normal / Behavior,Dog,Neutered Male,6.000000,Great Dane,1,black,Return to Owner


In [12]:
df_test = clean_data(df_test)
df_test.head()

dropped columns


/var/folders/yf/n0zzld6945vfyf6hj9_n1g4c0000gn/T/ipykernel_37174/850107665.py:4: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  dt_series = pd.to_datetime(df['intake_time'], errors='coerce')


cleaned intake time
cleaned intake condition
cleaned age and sex
cleaned breed
cleaned color


,intake_time,intake_type,intake_condition,animal_type,sex_upon_intake,age_upon_intake,breed,outcome_type,is_mix,primary_color
0,-9223372037,Normal,Dog,Neutered Male,2 years,None,Tricolor,NaN,0,1
1,-9223372037,Sick,Cat,Intact Female,4 weeks,None,Calico,NaN,0,9
2,-9223372037,Normal,Dog,Neutered Male,4 years,None,Tan/Gray,NaN,0,6
3,-9223372037,Normal,Dog,Intact Female,5 months,None,Brown/White,NaN,0,1
4,-9223372037,Injured,Cat,Intact Female,2 years,None,Black/White,NaN,0,2


In [123]:
# Transformer code for pipelines

# Use FunctionTransformer to wrap the freq_encode function
def apply_freq_encode(df):
    df = freq_encode(df, 'primary_color')
    df = freq_encode(df, 'breed')

    return df

In [ ]:
## PREDICTION STITCHING ##
def combine_predictions(dog_pred, cat_pred, dog_true, cat_true):
    """
    Pass in list of dog predictions and cat predictions
    Returns -> stitched together predictions based on original test set order
    """
    dog_pred_series = pd.Series(dog_pred, index=dog_true.index)
    cat_pred_series = pd.Series(cat_pred, index=cat_true.index)
    # Concatenate both series and sort by the original index so the output 
    # reflects the same order as the original test dataset
    all_predictions = pd.concat([dog_pred_series, cat_pred_series]).sort_index()
    final_df = pd.DataFrame({'Predicted_Label': all_predictions})
    csv_path = './test_predictions_combined.csv'
    final_df.to_csv(csv_path, index=False)
    print(f"Combined test predictions saved to: {csv_path}")

NameError: name 'dog_predictions' is not defined

In [ ]:
print('Done running ml_project.ipynb.')